## Include libraries

In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sksurv.column import encode_categorical

## data preprocessing

In [2]:
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper

def data_processing(data_df, fearure_list, top_n):
    data_df_x = data_df.drop(['LOC', 'UID', 'Mortality', 'SurvivalDays'], axis=1)

    data_df_y = data_df[['Mortality', 'SurvivalDays']]
    
    # feature selection
    top_n_fs = feature_list.features[0:top_n].values
    data_df_x = data_df_x[top_n_fs]  

    X_temp = data_df_x[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    y_temp = data_df_y[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    X_df_train, X_df_val, y_df_train, y_df_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=369)

    X_df_test_kao = data_df_x[data_df.LOC == '8']
    y_df_test_kao = data_df_y[data_df.LOC == '8']

    categorical_columns = ['Sex', 'AF', 'DM', 'HTN', 'Dyslipidemia', 'CHF', 'Smoking',
                           'Cancer.before.adm']
    
    categorical_columns = [x for x in categorical_columns if x in top_n_fs]
    
    numerical_columns = np.setdiff1d(data_df_x.columns, categorical_columns).tolist()

    categorical_ix = [data_df_x.columns.get_loc(col) for col in categorical_columns]
    numerical_ix =  np.setdiff1d(list(range(0, len(data_df_x.columns))), categorical_ix).tolist()

    scaler = preprocessing.StandardScaler()

    standardize = [([col], scaler) for col in numerical_columns]
    leave = [(col, None) for col in categorical_columns]

    x_mapper = DataFrameMapper(standardize + leave)

    X_df_train = pd.DataFrame(data=x_mapper.fit_transform(X_df_train),
                              columns=numerical_columns+categorical_columns,
                              index=X_df_train.index)

    X_df_val = pd.DataFrame(data=x_mapper.fit_transform(X_df_val),
                            columns=numerical_columns+categorical_columns,
                            index=X_df_val.index)

    X_df_test_kao = pd.DataFrame(data=x_mapper.fit_transform(X_df_test_kao),
                                 columns=numerical_columns+categorical_columns,
                                 index=X_df_test_kao.index)

    X_df_train = encode_categorical(X_df_train, columns=categorical_columns)
    X_df_val = encode_categorical(X_df_val, columns=categorical_columns)
    X_df_test_kao = encode_categorical(X_df_test_kao, columns=categorical_columns)
    
    return X_df_train, X_df_val, y_df_train, y_df_val, X_df_test_kao, y_df_test_kao

## DL setting

In [3]:
import torch
import torchtuples as tt
from pycox.evaluation import EvalSurv

get_target = lambda df: (df['SurvivalDays'].values, df['Mortality'].values)

## Feature selection

In [4]:
data = pd.read_csv(os.path.join('..', '..', 'data', '(v3_2)STROKE_VITAL_SIGN_MICE.csv'))
feature_list = pd.read_csv(os.path.join('..', '..', 'data', '(v3_2)ranked_list.csv'))

X_train_5, X_val_5, y_train, y_val, X_test_kao_5, y_test_kao = data_processing(data, feature_list, 5)
X_train_10, X_val_10, y_train, y_val, X_test_kao_10, y_test_kao = data_processing(data, feature_list, 10)
X_train_15, X_val_15, y_train, y_val, X_test_kao_15, y_test_kao = data_processing(data, feature_list, 15)

# for DLs
X_test_kao_5 = X_test_kao_5.values.astype('float32')
X_test_kao_10 = X_test_kao_10.values.astype('float32')
X_test_kao_15 = X_test_kao_15.values.astype('float32')
durations_test_kao, events_test_kao = get_target(y_test_kao)
# for RSF
y_train_cox =  y_train.copy()
y_train_cox.loc[:, 'Mortality'] = y_train_cox['Mortality'].astype(bool)
y_train_cox = np.array(list(y_train_cox.to_records(index=False)))
y_test_kao_cox =  y_test_kao.copy()
y_test_kao_cox.loc[:, 'Mortality'] = y_test_kao_cox['Mortality'].astype(bool)
y_test_kao_cox = np.array(list(y_test_kao_cox.to_records(index=False)))

## Deepsur

In [5]:
from pycox.models import CoxPH

def Deepsur(Xtrain, Xval, Ytrain, Yval):
    # preprocessing data
    Xtrain = Xtrain.values.astype('float32')
    Xval = Xval.values.astype('float32')
    Ytrain = get_target(Ytrain)
    Yval = get_target(Yval)
    val = Xval, Yval
    
    # parameters
    in_features = Xtrain.shape[1]
    num_nodes = [46, 32, 8]
    out_features = 1
    batch_norm = True
    dropout = 0.1
    output_bias = False
    batch_size = 256
    if Xtrain.shape[0]%batch_size == 1:
        batch_size = batch_size - 1
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=20)]
    verbose = False
    
    # network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)
    model = CoxPH(net, tt.optim.Adam)
    
    # train
    lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
    best_lr = lrfinder.get_best_lr()
    #_ = lrfinder.plot()
#     best_lr = 0.089
    model.optimizer.set_lr(best_lr)
    log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
                    val_data=val, val_batch_size=batch_size)
    return model
    

### top5-deepSur

In [6]:
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_sur = Deepsur(X_train_5, X_val_5, y_train, y_val)

In [7]:
# prediction
_ = deep_sur.compute_baseline_hazards()
surv_kao = deep_sur.predict_surv_df(X_test_kao_5)
# evaluation
ev_kao = EvalSurv(surv_kao, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.789


### top10-deepSur

In [8]:
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_sur = Deepsur(X_train_10, X_val_10, y_train, y_val)

In [9]:
# prediction
_ = deep_sur.compute_baseline_hazards()
surv_kao = deep_sur.predict_surv_df(X_test_kao_10)
# evaluation
ev_kao = EvalSurv(surv_kao, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.806


### top15-deepSur

In [10]:
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_sur = Deepsur(X_train_15, X_val_15, y_train, y_val)

In [11]:
# prediction
_ = deep_sur.compute_baseline_hazards()
surv_kao = deep_sur.predict_surv_df(X_test_kao_15)
# evaluation
ev_kao = EvalSurv(surv_kao, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.810


## DeepHit

In [22]:
from pycox.models import DeepHitSingle

def Deephit(Xtrain, Xval, Ytrain, Yval):
    # preprocessing data
    Xtrain = Xtrain.values.astype('float32')
    Xval = Xval.values.astype('float32')
    
    # label transforms
    num_durations = 480
    labtrans = DeepHitSingle.label_transform(num_durations)
    Ytrain = labtrans.fit_transform(*get_target(Ytrain))
    Yval = labtrans.transform(*get_target(Yval))
    train = (Xtrain, Ytrain)
    val = (Xval, Yval)
    
    # parameters
    in_features = Xtrain.shape[1]
    num_nodes = [46, 32, 8]
    out_features = labtrans.out_features
    batch_norm = True
    dropout = 0.1
    output_bias = False
    batch_size = 256
    if Xtrain.shape[0]%batch_size == 1:
        batch_size = batch_size - 1
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=20)]
    verbose = False
    
    # network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)
    
    model = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)

    lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
    best_lr = lrfinder.get_best_lr()
    #print(best_lr)
    #_ = lrfinder.plot()
    #best_lr = 0.089
    model.optimizer.set_lr(best_lr)
    
    log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
                    val_data=val, val_batch_size=batch_size)
    
    return model

### top-5 deepHit

In [23]:
# Seed
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_hit = Deephit(X_train_5, X_val_5, y_train, y_val)

In [24]:
# prediction
hit_surv = deep_hit.predict_surv_df(X_test_kao_5)
# evaluation
ev_kao = EvalSurv(hit_surv, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.790


### top-10 deepHit

In [25]:
# Seed
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_hit = Deephit(X_train_10, X_val_10, y_train, y_val)

In [26]:
# prediction
hit_surv = deep_hit.predict_surv_df(X_test_kao_10)
# evaluation
ev_kao = EvalSurv(hit_surv, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.811


### top-15 deepHit

In [27]:
# Seed
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_hit = Deephit(X_train_15, X_val_15, y_train, y_val)

In [28]:
# prediction
hit_surv = deep_hit.predict_surv_df(X_test_kao_15)
# evaluation
ev_kao = EvalSurv(hit_surv, durations_test_kao, events_test_kao, censor_surv='km')
print('Kao C-index = %.3f' %(ev_kao.concordance_td()))

Kao C-index = 0.810


## RandomSurvivalForest

### top-5 RF

In [19]:
from sksurv.ensemble import RandomSurvivalForest

rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)

rsf.fit(X_train_5, y_train_cox)
print('Kao C-index = %.3f' %(rsf.score(X_test_kao_5, y_test_kao_cox)))

Kao C-index = 0.782


### top-10 RF

In [20]:
from sksurv.ensemble import RandomSurvivalForest

rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)

rsf.fit(X_train_10, y_train_cox)
print('Kao C-index = %.3f' %(rsf.score(X_test_kao_10, y_test_kao_cox)))

Kao C-index = 0.805


### top-32 RF

In [21]:
from sksurv.ensemble import RandomSurvivalForest

rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)

rsf.fit(X_train_15, y_train_cox)
print('Kao C-index = %.3f' %(rsf.score(X_test_kao_15, y_test_kao_cox)))

Kao C-index = 0.810
